In [1]:
from ctapipe.core import Tool, Component
from traitlets import Integer, Float
import logging

In [2]:
class MyComponent(Component):
    name="mycomponent"
    description = "Do some things"

    value = Integer(default_value=-1, help="Value to use").tag(config=True)

class MyTool(Tool):
    name="mytool"
    description="do some things"

    def initialize():

In [3]:
tool = MyTool()
comp = MyComponent(tool)
comp2 = MyComponent(tool, name="comp2")

In [4]:
tool.log.setLevel('DEBUG')
comp.parent.log.info("This is a test")
comp.log.info("popop")
comp2.log.info("sdsds")

[MyTool] This is a test
[MyTool.mycomponent] popop
[MyTool.comp2] sdsds


In [5]:
tool.print_help(classes=True)

do some things

Options
-------

Arguments that take values are actually convenience aliases to full
Configurables, whose aliases are listed on the help line. For more information
on full configurables, see '--help-all'.

--log-level=<Enum> (Application.log_level)
    Default: 30
    Choices: (0, 10, 20, 30, 40, 50, 'DEBUG', 'INFO', 'WARN', 'ERROR', 'CRITICAL')
    Set the log level by value or name.

Class parameters
----------------

Parameters are set from command-line arguments of the form:
`--Class.trait=value`. This line is evaluated in Python, so simple expressions
are allowed, e.g.:: `--C.a='range(3)'` For setting C.a=[0,1,2].

MyTool options
--------------
--MyTool.log_datefmt=<Unicode>
    Default: '%Y-%m-%d %H:%M:%S'
    The date format used by logging formatters for %(asctime)s
--MyTool.log_format=<Unicode>
    Default: '[%(name)s]%(highlevel)s %(message)s'
    The Logging format template
--MyTool.log_level=<Enum>
    Default: 30
    Choices: (0, 10, 20, 30, 40, 50, 'DEBUG'

In [6]:
l = logging.getLogger("boop")

In [7]:
l.getChild?